# DataBase connection

In [ ]:
# Run below commands in google colab
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.0
!wget -q http://apache.osuosl.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
# unzip it
!tar xf spark-2.4.8-bin-hadoop2.7.tgz
# install findspark 
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"
import findspark
findspark.init()

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession \
.builder \
.appName('MasterDB') \
.config("spark.mongodb.connection.uri", "mongodb+srv://*************************************************") \
.config('spark.jar.packages', 'org.mongodb.spark:mongo-spark-connector_2.11:2.4.1') \
.enableHiveSupport() \
.getOrCreate()

In [ ]:
sc = SparkContext.getOrCreate()

Inserire nell'output del codice qui di seguito i comandi commentati. Dopo il completamento delle operazioni, rinominare il file json presente nella cartella Sandwitch3.json con il nome di: 'data.json'.
Fermare l'esecuzione del comando e proseguire con l'esecuzione dei comandi successivi

In [ ]:
#df = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri", "mongodb+srv://**********************************************************").load()
#df.coalesce(1).write.format('json').save('/content/Sandwitch3.json')
!$SPARK_HOME/bin/pyspark --packages org.mongodb.spark:mongo-spark-connector_2.11:2.4.1

Python 3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/content/spark-2.4.8-bin-hadoop2.7/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.mongodb.spark#mongo-spark-connector_2.11 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a5774794-c8b8-424a-9953-2294fb580a58;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.11;2.4.1 in central
	found org.mongodb#mongo-java-driver;3.10.2 in central
	[3.10.2] org.mongodb#mongo-java-driver;[3.10,3.11)
downloading https://repo1.maven.org/maven2/org/mongodb/spark/mongo-spark-connector_2.11/2.4.1/mongo-spark-connector_2.11-2.4.1.jar ...
	[SUCCESSFUL ] org.mongodb.spark#mongo-spark-connector_2.11;2.4.1!mongo-spark-connector_2.11.jar (86ms)
downloading https

# Data Preparation

In [ ]:
df2 = spark.read.json('/content/Sandwitch3.json/data.json')

In [ ]:
import re
import string
def remove_punct(text):
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')
    nopunct = regex.sub(" ", text)
    return nopunct

def convert_rating(rating):
    rating = int(rating)
    if rating >=4: return 2
    elif rating == 3: return 1
    else: return 0

In [ ]:
from pyspark.sql.functions import *
punct_remover = udf(lambda x: remove_punct(x))
rating_convert = udf(lambda x: convert_rating(x))
review_df = df2.select('review_id', punct_remover('text'), rating_convert('stars'))

In [ ]:
from pyspark.sql.types import IntegerType
review_df = review_df.withColumnRenamed('<lambda>(text)', 'text')\
                     .withColumn('label', review_df["<lambda>(stars)"].cast(IntegerType()))\
                     .drop('<lambda>(stars)')

In [ ]:
from pyspark.ml.feature import *
tok = Tokenizer(inputCol="text", outputCol="words")
review_tokenized = tok.transform(review_df)

In [ ]:
# remove stop words
stopword_rm = StopWordsRemover(inputCol='words', outputCol='words_nsw')
review_tokenized = stopword_rm.transform(review_tokenized)

In [ ]:
review_tokenized.show()

+--------------------+--------------------+-----+--------------------+--------------------+
|           review_id|                text|label|               words|           words_nsw|
+--------------------+--------------------+-----+--------------------+--------------------+
|KU_O5udG6zpxOg-Vc...|If you decide to ...|    1|[if, you, decide,...|[decide, eat, , a...|
|Sx8TMOWLNuJBWer-0...|Cute interior and...|    2|[cute, interior, ...|[cute, interior, ...|
|OAhBYw8IQ6wlfw1ow...|Great place for b...|    2|[great, place, fo...|[great, place, br...|
|40thYphUgIfvJq17Q...|Great staff alway...|    2|[great, staff, al...|[great, staff, al...|
|L0jv8c2FbpWSlfNC6...|What a great addi...|    2|[what, a, great, ...|[great, addition,...|
|JBWZmBy69VMggxj3e...|My boyfriend and ...|    2|[my, boyfriend, a...|[boyfriend, tried...|
|qGQvUHmC02PAZW3H1...|Stopped by after ...|    2|[stopped, by, aft...|[stopped, sunday,...|
|A4n4YaE-owOVgTQcr...|The cafe was extr...|    2|[the, cafe, was, ...|[cafe, ext

In [ ]:
%cd /content/spark-2.4.8-bin-hadoop2.7/jars
!wget https://repo1.maven.org/maven2/org/datasyslab/geotools-wrapper/1.1.0-25.2/geotools-wrapper-1.1.0-25.2.jar
!wget https://repo1.maven.org/maven2/org/apache/sedona/sedona-python-adapter-3.0_2.12/1.2.0-incubating/sedona-python-adapter-3.0_2.12-1.2.0-incubating.jar
!wget https://repo1.maven.org/maven2/org/apache/sedona/sedona-viz-3.0_2.12/1.2.0-incubating/sedona-viz-3.0_2.12-1.2.0-incubating.jar

/content/spark-2.4.8-bin-hadoop2.7/jars
--2022-06-19 06:58:15--  https://repo1.maven.org/maven2/org/datasyslab/geotools-wrapper/1.1.0-25.2/geotools-wrapper-1.1.0-25.2.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29767549 (28M) [application/java-archive]
Saving to: ‘geotools-wrapper-1.1.0-25.2.jar’

geotools-wrapper-1. 100%[===================>]  28.39M   154MB/s    in 0.2s    

2022-06-19 06:58:15 (154 MB/s) - ‘geotools-wrapper-1.1.0-25.2.jar’ saved [29767549/29767549]

--2022-06-19 06:58:15--  https://repo1.maven.org/maven2/org/apache/sedona/sedona-python-adapter-3.0_2.12/1.2.0-incubating/sedona-python-adapter-3.0_2.12-1.2.0-incubating.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP reques

In [ ]:
df2.show()

+--------------------+--------------------+--------------------+-----+-----+--------------------+
|                 _id|          categories|           review_id|stars|state|                text|
+--------------------+--------------------+--------------------+-----+-----+--------------------+
|[62adaa1578d7c816...|Restaurants, Brea...|KU_O5udG6zpxOg-Vc...|    3|   PA|If you decide to ...|
|[62adaa1578d7c816...|Sandwiches, Beer,...|Sx8TMOWLNuJBWer-0...|    4|   LA|Cute interior and...|
|[62adaa1578d7c816...|Sandwiches, Resta...|OAhBYw8IQ6wlfw1ow...|    5|   FL|Great place for b...|
|[62adaa1578d7c816...|Food, Coffee & Te...|40thYphUgIfvJq17Q...|    5|   PA|Great staff alway...|
|[62adaa1578d7c816...|Food, Restaurants...|L0jv8c2FbpWSlfNC6...|    5|   CA|What a great addi...|
|[62adaa1578d7c816...|Restaurants, Deli...|JBWZmBy69VMggxj3e...|    5|   PA|My boyfriend and ...|
|[62adaa1578d7c816...|Bagels, Sporting ...|qGQvUHmC02PAZW3H1...|    5|   TN|Stopped by after ...|
|[62adaa1578d7c816..

In [ ]:
df2 = df2.drop('_id')
df2 = df2.drop('categories')
df2 = df2.drop('stars')
df2 = df2.drop('text')

In [ ]:
df2.show()

+--------------------+-----+
|           review_id|state|
+--------------------+-----+
|KU_O5udG6zpxOg-Vc...|   PA|
|Sx8TMOWLNuJBWer-0...|   LA|
|OAhBYw8IQ6wlfw1ow...|   FL|
|40thYphUgIfvJq17Q...|   PA|
|L0jv8c2FbpWSlfNC6...|   CA|
|JBWZmBy69VMggxj3e...|   PA|
|qGQvUHmC02PAZW3H1...|   TN|
|A4n4YaE-owOVgTQcr...|   LA|
|mCVyppNwK2qPOJHXg...|   PA|
|Ad91EzbAGRCR1SwkP...|   IL|
|Lk21QNbrI_e3nf0Gv...|   ID|
|5obXxR0b94b5q6j1z...|   CA|
|RGV9GWhAAfAAlYyd4...|   PA|
|vZNDuj4a8cFQV9t6-...|   PA|
|S4nZgOgiv9w8MOiaW...|   TN|
|zqmkEnp1kfU2vosDc...|   LA|
|pJRn59F_lyNO1zT3T...|   TN|
|d9j7KC7FTOkoI_L-i...|   PA|
|xilt6zGZ3Nqo2_fog...|   MO|
|qsjMCjaEW9UWyzAw4...|   PA|
+--------------------+-----+
only showing top 20 rows



In [ ]:
splits = review_tokenized.randomSplit([0.7,0.3],seed=100)
train = splits[0]
test = splits[1]

In [ ]:
from pyspark.ml import Pipeline
n = 3
def build_trigrams(inputCol=["words_nsw"], n=3):
    ngrams = [
        NGram(n=i, inputCol="words_nsw", outputCol="{0}_grams".format(i))
        for i in range(1, n + 1)
    ]
    cv = [
          CountVectorizer(vocabSize=2**14,inputCol="{0}_grams".format(i),
              outputCol="{0}_tf".format(i))
          for i in range(1, n + 1)
      ]
    idf = [IDF(inputCol="{0}_tf".format(i), outputCol="{0}_tfidf".format(i), minDocFreq=5) for i in range(1, n + 1)]
 
    assembler = [VectorAssembler(
        inputCols=["{0}_tfidf".format(i) for i in range(1, n + 1)],
        outputCol="features"
    )]
    return Pipeline(stages=ngrams + cv + idf + assembler)

trigrams_builder = build_trigrams().fit(train)

In [ ]:
train_f = trigrams_builder.transform(train)

In [ ]:
test_f = trigrams_builder.transform(test)

In [ ]:
train_f = train_f.select(['review_id','features','label']).cache()
test_f = test_f.select(['review_id','features','label']).cache()

In [ ]:
train_f.show()

+--------------------+--------------------+-----+
|           review_id|            features|label|
+--------------------+--------------------+-----+
|--1qZGjv7QmhOl6oM...|(49152,[0,5,9,10,...|    0|
|--2p5FajyhBVDP4mE...|(49152,[0,9,16,40...|    0|
|--31YAQSS12djWLzr...|(49152,[0,7,9,12,...|    0|
|--6LVUGf9g8jAnJLC...|(49152,[0,12,15,2...|    2|
|--7P9vyDUYRTlGN5j...|(49152,[0,2,3,5,1...|    2|
|--7rYd13Xc971wy3U...|(49152,[0,2,5,7,8...|    2|
|--LY07gmzbdJqYH3E...|(49152,[0,2,5,13,...|    2|
|--MH7M0oeKjPtEtV9...|(49152,[0,2,12,76...|    0|
|--NVZE6Zpshs5Z0jR...|(49152,[0,1,5,44,...|    0|
|--OoCf3YF_dn76-SW...|(49152,[0,11,17,1...|    1|
|--R9gPPZpRwk14nkY...|(49152,[0,2,5,11,...|    1|
|--RrOZv-5KewKALtS...|(49152,[0,1,2,5,1...|    2|
|--SLCuhlm58GrBcru...|(49152,[0,1,3,6,7...|    0|
|--VQPtRVXzarTPDZ2...|(49152,[0,5,9,10,...|    0|
|--YCb3I1Uz6m5CViY...|(49152,[0,4,5,8,1...|    0|
|--ZLBO8pffrdZA_KD...|(49152,[0,2,3,8,1...|    1|
|--ZpMyndwYdmxHfnV...|(49152,[0,2,6,8,1...|    2|


# Model Selection

SVM

In [ ]:
from pyspark.ml.classification import RandomForestClassifier, LinearSVC
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import  OneVsRest

svm = LinearSVC()
ovr = OneVsRest(classifier=svm, labelCol='label', featuresCol='features')

# SVM
crossval = CrossValidator(estimator=ovr,
                          estimatorParamMaps=ParamGridBuilder().build(),
                          evaluator=MulticlassClassificationEvaluator(metricName='accuracy'),
                          numFolds=5,
                          parallelism=4)

In [ ]:
bestmodel= crossval.fit(train_f)

In [ ]:
print(f'Best accuracy -> {bestmodel.avgMetrics[0]}')

Best accuracy -> 0.8599848415745013


RANDOM FOREST

In [ ]:
crossval2 = CrossValidator(estimator=RandomForestClassifier(),
                          estimatorParamMaps=ParamGridBuilder().build(),
                          evaluator=MulticlassClassificationEvaluator(metricName='accuracy'),
                          numFolds=5,
                          parallelism=4)

In [ ]:
bestmodel2 = crossval2.fit(train_f)

In [ ]:
print(f'Best accuracy -> {bestmodel2.avgMetrics[0]}')

Best accuracy -> 0.6976260625254567


NAIVE BAYES

In [ ]:
from pyspark.ml.classification import NaiveBayes, NaiveBayesModel

nb = NaiveBayes()
ovr = OneVsRest(classifier=nb, labelCol='label', featuresCol='features')

# NB
crossval3 = CrossValidator(estimator=ovr,
                          estimatorParamMaps=ParamGridBuilder().build(),
                          evaluator=MulticlassClassificationEvaluator(metricName='accuracy'),
                          numFolds=5,
                          parallelism=4)


In [ ]:
bestmodel3 = crossval3.fit(train_f)

In [ ]:
print(f'Best accuracy -> {bestmodel3.avgMetrics[0]}')

Best accuracy -> 0.8132377344548078


LOGISTIC REGRESSION

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression()
ovr = OneVsRest(classifier=lr, labelCol='label', featuresCol='features')

# LR
crossval4 = CrossValidator(estimator=ovr,
                          estimatorParamMaps=ParamGridBuilder().build(),
                          evaluator=MulticlassClassificationEvaluator(metricName='accuracy'),
                          numFolds=5,
                          parallelism=4)

In [ ]:
bestmodel4 = crossval4.fit(train_f)

In [ ]:
print(f'Best accuracy -> {bestmodel4.avgMetrics[0]}')

Best accuracy -> 0.8217810893985267


# Evaluation

In [ ]:
predictions = bestmodel.transform(test_f)

In [ ]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.8538930342623292

#Analisi Risultati

In [ ]:
predictions.show()

+--------------------+--------------------+-----+----------+
|           review_id|            features|label|prediction|
+--------------------+--------------------+-----+----------+
|--6ROZMJu22IL-xTp...|(49152,[0,3,9,12,...|    2|       2.0|
|--9C_kg5WRVuJMqHj...|(49152,[0,1,3,10,...|    0|       0.0|
|--DVI2tJfuL83Kxtn...|(49152,[0,4,6,11,...|    2|       2.0|
|--ErNWkQ_WikueeIx...|(49152,[0,4,14,24...|    0|       2.0|
|--RjvM2Gn3mWoQwhS...|(49152,[0,5,17,22...|    2|       2.0|
|--UE7GSquo0AbTl8k...|(49152,[0,3,19,27...|    2|       2.0|
|--UnH3qXADKVzgqdC...|(49152,[0,1,2,5,1...|    2|       2.0|
|--WOjztZDfS7PrGIr...|(49152,[0,1,3,5,8...|    2|       2.0|
|--_MftZAFiD8-3o9X...|(49152,[0,5,16,26...|    2|       0.0|
|--cVB07L57FdknveQ...|(49152,[0,3,63,89...|    2|       2.0|
|--kkSP4cNwj69fWr_...|(49152,[0,3,4,12,...|    1|       0.0|
|--m5ksROYsBRjEvvJ...|(49152,[0,2,14,19...|    2|       2.0|
|--m9ha-OEYtZmTg6C...|(49152,[0,1,2,6,2...|    2|       2.0|
|--tmgErXL6dlDaV3n...|(4

In [ ]:
df2.withColumnRenamed("review_id","reviews")
pred = predictions.join(df2, predictions.review_id == df2.reviews, 'left')

In [ ]:
pred.show()

+--------------------+--------------------+-----+----------+--------------------+-----+
|           review_id|            features|label|prediction|           review_id|state|
+--------------------+--------------------+-----+----------+--------------------+-----+
|--zAqGUeMdjJ0aeX0...|(49152,[0,7,12,21...|    2|       2.0|--zAqGUeMdjJ0aeX0...|   TN|
|-8OCeO-WOJILhwrUX...|(49152,[0,3,8,9,1...|    2|       2.0|-8OCeO-WOJILhwrUX...|   MO|
|-A9KRZAjatnKIKGkZ...|(49152,[0,3,5,15,...|    2|       2.0|-A9KRZAjatnKIKGkZ...|   PA|
|-AQF4jRN2EMz5KIOW...|(49152,[0,1,2,3,4...|    2|       2.0|-AQF4jRN2EMz5KIOW...|   TN|
|-GehB4C8_DeDOjToM...|(49152,[0,1,2,3,4...|    2|       2.0|-GehB4C8_DeDOjToM...|   IN|
|-Jp67xnF9f9-Y41KE...|(49152,[0,3,7,8,1...|    0|       0.0|-Jp67xnF9f9-Y41KE...|   PA|
|-NQInBApLOBqVhqNM...|(49152,[0,2,4,6,1...|    2|       2.0|-NQInBApLOBqVhqNM...|   FL|
|-OsotBwMDsZNlM_Zg...|(49152,[0,8,21,26...|    2|       2.0|-OsotBwMDsZNlM_Zg...|   PA|
|-SQXDf-c0aD6BHtqo...|(49152,[0,

In [ ]:
pred.show()

+--------------------+--------------------+-----+----------+--------------------+-----+
|           review_id|            features|label|prediction|           review_id|state|
+--------------------+--------------------+-----+----------+--------------------+-----+
|--zAqGUeMdjJ0aeX0...|(49152,[0,7,12,21...|    2|       2.0|--zAqGUeMdjJ0aeX0...|   TN|
|-8OCeO-WOJILhwrUX...|(49152,[0,3,8,9,1...|    2|       2.0|-8OCeO-WOJILhwrUX...|   MO|
|-A9KRZAjatnKIKGkZ...|(49152,[0,3,5,15,...|    2|       2.0|-A9KRZAjatnKIKGkZ...|   PA|
|-AQF4jRN2EMz5KIOW...|(49152,[0,1,2,3,4...|    2|       2.0|-AQF4jRN2EMz5KIOW...|   TN|
|-GehB4C8_DeDOjToM...|(49152,[0,1,2,3,4...|    2|       2.0|-GehB4C8_DeDOjToM...|   IN|
|-Jp67xnF9f9-Y41KE...|(49152,[0,3,7,8,1...|    0|       0.0|-Jp67xnF9f9-Y41KE...|   PA|
|-NQInBApLOBqVhqNM...|(49152,[0,2,4,6,1...|    2|       2.0|-NQInBApLOBqVhqNM...|   FL|
|-OsotBwMDsZNlM_Zg...|(49152,[0,8,21,26...|    2|       2.0|-OsotBwMDsZNlM_Zg...|   PA|
|-SQXDf-c0aD6BHtqo...|(49152,[0,

In [ ]:
prov = pred.groupBy('state').sum('prediction')

In [ ]:
prov.show()

+-----+---------------+
|state|sum(prediction)|
+-----+---------------+
|   AZ|        17050.0|
|   LA|        45322.0|
|   NJ|        10400.0|
|   NV|        12859.0|
|   ID|         7644.0|
|   CA|        14346.0|
|   DE|         2931.0|
|   MO|        24393.0|
|   IL|         1806.0|
|   IN|        21807.0|
|   TN|        27923.0|
|   PA|        76328.0|
|   AB|         2341.0|
|   FL|        51801.0|
+-----+---------------+



In [ ]:
states = prov.drop('sum(prediction)')

In [ ]:
states.show()

+-----+
|state|
+-----+
|   AZ|
|   LA|
|   NJ|
|   NV|
|   ID|
|   CA|
|   DE|
|   MO|
|   IL|
|   IN|
|   TN|
|   PA|
|   AB|
|   FL|
+-----+



In [ ]:
for x in states.rdd.collect():
  print(x.state)
  p2 = pred.filter(col('state')==x.state)
  print(evaluator.evaluate(p2))


AZ
0.8525439855675269
LA
0.8541647167410583
NJ
0.8468776803841283
NV
0.8696125015308969
ID
0.8805453309026109
CA
0.8759665862685062
DE
0.8423410445412531
MO
0.8634459272200306
IL
0.8386812231011626
IN
0.8591614298669961
TN
0.855925863691454
PA
0.8353164758178029
AB
0.8214179131773786
FL
0.865691806082538
